# Make tables: Merge all cancer dfs and create csv files for interacting proteins

This notebook combines dfs with p-values and differential expressions for 8 cancers. 
Description of created csv files: 
* all_proteins - contains all data in normal format, 
* all_heatmap - all data appended to make a long table for easy use with heatmap function, 
* sig_pval_heatmap - contains only significant genes (long format),

Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u
import plot_utils as p

Read in the signle cancer dfs with p-values and differential expressions. Merge all dfs into one pancancer data frame.

In [2]:
l_merged = pd.read_csv('csv/Single_Cancer/LUAD_all_phospho.csv')
e_merged = pd.read_csv('csv/Single_Cancer/EC_all_phospho.csv')
c_merged = pd.read_csv('csv/Single_Cancer/CO_all_phospho.csv')

In [3]:
l_merged

,Phosphoproteomics,Luad_P_Value,Luad_Median
0,KRT8_S477,0.000079,2.82370
1,KRT8_S475S478,0.000079,4.51545
2,EEF1B2_S8,0.000079,2.11240
3,DNMBP_S1436,0.000244,1.63160
4,CTBP2_S905,0.000310,1.22980
...,...,...,...
39664,ZNF281_S864,NaN,0.07070
39665,ZNF462_S1747,NaN,-0.87780
39666,ZNF638_S1462,NaN,-1.10495
39667,ZNF687_S227,NaN,0.77530


# Make csv of all data

In [4]:
df1 = l_merged.merge(e_merged, on='Phosphoproteomics',how='outer')
all_df = df1.merge(c_merged, on='Phosphoproteomics',how='outer')

In [5]:
all_df.to_csv('csv/all_phospho.csv', index=False)

# Make csv of data formatted to use with the HeatMap function 

In [2]:
# Create long df for heatmap

cancer = ['LUAD','EC','CO']
merged_dfs = [l_merged,e_merged,c_merged]

all_long_df = pd.DataFrame()
i = 0
for c in cancer:
    m = merged_dfs[i]
    m2 = m.assign(Cancer = c)
    m2 = m2.rename(columns={c+'_P_Value': 'P_Value'})
    m2 = m2.rename(columns={c+'_Median': 'Medians'})
    all_long_df = all_long_df.append(m2) 
    if i < 7:
        i += 1

#print('Check total proteins:', len(all_long_df.Phospho.unique()))

NameError: name 'l_merged' is not defined

In [1]:
all_long_df.to_csv('csv/all_heatmap_phospho.csv', index=False)

NameError: name 'all_long_df' is not defined

# Make csv of only significant proteins (formatted for heatmap)

Read in list_sig_in_at_least_one_cancer.csv. Convert to a list of genes significant in at least 1 cancer. Slice out genes in the list from the df with all data.

In [ ]:
sig = pd.read_csv('csv/list_sig_all_phospho.csv')
list_sig = list(sig['0'])

In [ ]:
# Keep genes with at least one sig ttest
bool_df = all_long_df['Phospho'].isin(list_sig)
sig_df = all_long_df[bool_df]
print('Check total sig genes:', len(sig_df.Phospho.unique()))
t = list(sig_df.Phospho)

In [ ]:
sig_df.to_csv('csv/sig_pval_heatmap_phospho.csv', index=False)

# Multiple sig 

In [ ]:
mult_sig = pd.read_csv('csv/list_sig_mult_all_phospho.csv')
list_mult_sig = list(mult_sig['0'])

In [ ]:
# Keep genes with at least one sig ttest
bool_df = all_long_df['Phospho'].isin(list_mult_sig)
mult_df = all_long_df[bool_df]
print('Check total sig genes:', len(mult_df.Phospho.unique()))

In [ ]:
mult_df.to_csv('csv/mult_sig_heatmap_phospho.csv', index=False)